In [31]:
import numpy as np
import random
import pandas as pd
from neo4j import GraphDatabase

Crear Data Frame

In [32]:
#ID Unico de Transaccion
id_transaccion = []
for i in range(1, 50001):
    id_transaccion.append(i)

#print(id_transaccion)

#Numero de cliente/vendedor
numero_vendedor = np.random.randint(low = 1, high = 2500, size = (50000,) )

#Cantidad de transaccion USD
cant_transaccion = np.random.randint(low = 20, high = 1250000, size = (50000))

#Tipo de transaccion
tipo_transaccion =['retiro_efectivo', 'deposito_dinero', 'transferencia_domestica', 'transferencia_internacional']

#generar lista de enteros randoms
random_integers = []
for i in range(0, 50000):
    random_num = random.randint(0,3)
    random_integers.append(random_num)

lista_transaccion = []
for i in random_integers:
    lista_transaccion.append(tipo_transaccion[i])

data_transacciones = {
    "ID_Transaccion" : id_transaccion, #0 
    "Numero_Vendedor" : list(numero_vendedor), #1 
    "Cantidad_Transferida" : list(cant_transaccion), #2
    "Tipo_Transaccion" : lista_transaccion, #3
}

dataFrame_transacciones = pd.DataFrame(data_transacciones)

print(dataFrame_transacciones[:20])

    ID_Transaccion  Numero_Vendedor  Cantidad_Transferida  \
0                1              274                 59290   
1                2             1927               1062151   
2                3             1877                531716   
3                4             1722               1116309   
4                5             2175                426350   
5                6             1816               1220087   
6                7              746                636470   
7                8             1388                113343   
8                9              533                854395   
9               10               56                 59069   
10              11              639                106537   
11              12             1526                410327   
12              13              428                276713   
13              14             1986               1144352   
14              15             2100                302932   
15              16      

Crear y subir la DB a Neo4j

In [33]:


#comandos para escribir datos en la DB
lista_transaccion = dataFrame_transacciones.values.tolist()
comandos_ejecucion_transaccion = []

for i in lista_transaccion:
    #neo4j_create_statement = "create (t:Transaccion {id_transaccion:" + str(i[0]) + ", numero_vendedor: " + str(i[1]) + ", cant_transaccion: " + str(i[2]) + ", tipo_transaccion: " + str(i[3]) +  "'}) "

    neo4j_create_statement = "merge (t:Transaction {transaction_id:" + str(i[0]) + ", vendor_number:  " + str(i[1]) +", transaction_amount: " + str(i[2]) +", transaction_type: '" + str(i[3]) + "'})"

    comandos_ejecucion_transaccion.append(neo4j_create_statement)

#NEW PASSWORD:   password123
def ejecutar_transacciones(comandos_ejecucion_transaccion):
    conexion_DB = GraphDatabase.driver(uri = "bolt://localhost:7687", auth = ("neo4j", "password123") )
    session = conexion_DB.session()
    session.run('MATCH (n) DETACH DELETE n') #DELETE DATA
    print("DATA DELETED\n")
    for i in comandos_ejecucion_transaccion:
        session.run(i)

ejecutar_transacciones(comandos_ejecucion_transaccion)

print("ALL DONE!\n")

DATA DELETED

ALL DONE!



Data Query

In [37]:
def get_data(tx):
    result = tx.run("MATCH (t:Transaction) RETURN t.transaction_id AS transaction_id")
    transactions = list(result)  # a list of Record objects
    summary = result.consume()
    return transactions, summary

with GraphDatabase.driver(uri = "bolt://localhost:7687", auth=("neo4j", "password123") ) as driver:
    with driver.session(database="neo4j") as session:
        transactions, summary = session.execute_read(get_data)

        # Summary information
        print("The query `{query}` returned {transactions_count} transactions in {time} ms.".format(
            query=summary.query, transactions_count=len(transactions),
            time=summary.result_available_after
        ))

        # Loop through results and do something with them
        for transaccion in transactions:
            print(transaccion.data())  # obtain record as dict

The query `MATCH (t:Transaction) RETURN t.transaction_id AS transaction_id` returned 50000 transactions in 0 ms.
{'transaction_id': 46217}
{'transaction_id': 46218}
{'transaction_id': 46219}
{'transaction_id': 46220}
{'transaction_id': 46221}
{'transaction_id': 46222}
{'transaction_id': 46223}
{'transaction_id': 46224}
{'transaction_id': 46225}
{'transaction_id': 46226}
{'transaction_id': 46227}
{'transaction_id': 46228}
{'transaction_id': 46229}
{'transaction_id': 46230}
{'transaction_id': 46231}
{'transaction_id': 46232}
{'transaction_id': 46233}
{'transaction_id': 46234}
{'transaction_id': 46235}
{'transaction_id': 46236}
{'transaction_id': 46237}
{'transaction_id': 46238}
{'transaction_id': 46239}
{'transaction_id': 46240}
{'transaction_id': 46241}
{'transaction_id': 46242}
{'transaction_id': 46243}
{'transaction_id': 46244}
{'transaction_id': 46245}
{'transaction_id': 46246}
{'transaction_id': 46247}
{'transaction_id': 46248}
{'transaction_id': 46249}
{'transaction_id': 46250}
{'t

Update the database

In [42]:
def update_transaction(tx, vendor_number, transaction_amount, transaction_type):
    result = tx.run(
        "merge (t:Transaction {transaction_id:" + str(i[0]) + ", vendor_number:  " + str(i[1]) +", transaction_amount: " + str(i[2]) +", transaction_type: '" + str(i[3]) + "'})", vendor_number = vendor_number, 
        transaction_amount = transaction_amount, transaction_type = transaction_type

        #neo4j_create_statement = "merge (t:Transaction {transaction_id:" + str(i[0]) + ", vendor_number:  " + str(i[1]) +", transaction_amount: " + str(i[2]) +", transaction_type: '" + str(i[3]) + "'})"
    )
    summary = result.consume()
    return summary

with driver.session(database="neo4j") as session:
    summary = session.execute_write(update_transaction, vendor_number="1999", transaction_amount=99999, transaction_type='transferencia_internacional')
    print(f"Query counters: {summary.counters}.")
    print("DONE")



#def update_person(tx, name, age):
#    result = tx.run("""
#        MATCH (p:Person {name: $name})
#        SET p.age = $age
#        """, name=name, age=age
#    )
#    summary = result.consume()
#    return summary


Query counters: {}.
